# D. Modelling<a id = 'C_Pre-processing'></a>

# 1. Contents<a id='1._Contents'></a>
* [D. Modelling](#C_Pre-processing)
* [2. Objective](#2_Objective)
    * [2.1 Tasks](#2.1_Tasks)
* [3. Imports](#3_Imports)
* [4. Load Data](#4._Load_Data)
* [5. Revising and Creating New Features](#5._Revising_and_Creating_New_Features)
    * [5.1 Revising the yearBuilt feature](#5.1_Revising_the_yearBuilt_feature)
    * [5.2 Creating a new feature for state rent clusters](#5.2_Creating_a_new_feature_for_state_rent_clusters)
* [6. Create a Dataframe for the features and target variable](#6._Create_a_Dataframe_for_the_features_and_target_variable)
* [7. Develop an Imputation Strategy](#7._Develop_an_Imputation_Strategy)
* [8. Create Train-Test Split](#8._Create_Train-Test_Split)
* [9. Create One Hot Encoding Object](#9._Create_One_Hot_Encoding_Object)
* [10. Create Scaling Object](#10._Create_Scaling_Object)
* [11. Modelling](#11._Modelling)
* [12. Assessing Models by Pre-selecting Features](#12._Assessing_Models_by_Pre-selecting_Features)
* [13.Hyper parameter tuning on ridge learner](#13._Hyper_parameter_tuning_on_ridge_learner)
* [14. XGBoost Model](#14._XGBoost_Model)
* [15. Model Selection ](#15._Model_Selection)
* [16. Save Best Model](#16._Save_best_model)

# 2. Objective<a id = '2_Objective'></a>

- Create a predictive model for rentals data

# 3. Imports<a id = '3_Imports'></a>

In [274]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from datetime import datetime as dt
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.feature_selection import SelectKBest, f_regression,SelectPercentile
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn_pandas import DataFrameMapper
from sklearn.pipeline import FeatureUnion
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import sklearn.metrics
import eli5
import pickle

# 4. Load Data<a id='4._Load_Data'></a>

In [2]:
read = pd.read_csv('..\data\interim\\rentals_process.csv', parse_dates = ['yearConstructed', 'date'])

In [3]:
rentals = read.copy()

In [4]:
rentals.shape

(266220, 37)

In [5]:
rentals.head()

,state,serviceCharge,heatingType,telekomTvOffer,newlyConst,balcony,telekomUploadSpeed,yearConstructed,firingTypes,hasKitchen,...,sqm_per_room,area_km2,population_2019,population_per_km2,gdp_per_capita_2018,hdi,total_state_listings,total_state_sqm,listings_per_1000capita,listings_per_100sqm
0,Nordrhein_Westfalen,245.0,central_heating,ONE_YEAR_FREE,False,False,10.0,1965-01-01,oil,False,...,21.50,34085,17932651,526,39678,0.936,62069,4615660.71,3.461228,1.344748
1,Nordrhein_Westfalen,95.0,self_contained_central_heating,ONE_YEAR_FREE,False,False,40.0,1953-01-01,gas,False,...,24.00,34085,17932651,526,39678,0.936,62069,4615660.71,3.461228,1.344748
2,Nordrhein_Westfalen,200.0,central_heating,ONE_YEAR_FREE,False,False,40.0,1951-01-01,oil,False,...,30.86,34085,17932651,526,39678,0.936,62069,4615660.71,3.461228,1.344748
3,Nordrhein_Westfalen,215.0,gas_heating,ONE_YEAR_FREE,True,True,2.4,2018-01-01,gas,False,...,29.00,34085,17932651,526,39678,0.936,62069,4615660.71,3.461228,1.344748
4,Nordrhein_Westfalen,121.0,central_heating,ONE_YEAR_FREE,False,True,40.0,1914-01-01,gas,False,...,26.00,34085,17932651,526,39678,0.936,62069,4615660.71,3.461228,1.344748


In [6]:
rentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266220 entries, 0 to 266219
Data columns (total 37 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   state                    266220 non-null  object        
 1   serviceCharge            259490 non-null  float64       
 2   heatingType              221782 non-null  object        
 3   telekomTvOffer           233938 non-null  object        
 4   newlyConst               266220 non-null  bool          
 5   balcony                  266220 non-null  bool          
 6   telekomUploadSpeed       233208 non-null  float64       
 7   yearConstructed          209427 non-null  datetime64[ns]
 8   firingTypes              209806 non-null  object        
 9   hasKitchen               266220 non-null  bool          
 10  cellar                   266220 non-null  bool          
 11  rent                     266220 non-null  float64       
 12  livingSpace     

# 5. Revising and Creating New Features<a id='5._Revising_and_Creating_New_Features'></a>

## 5.1 Revising the yearBuilt feature<a id='5.1_Revising_the_yearBuilt_features'></a>

In [7]:
#Convert yearBuilt feature to numeric format
rentals['yearBuilt'] = rentals.yearConstructed.dt.year

## 5.2 Creating a new feature for state rent clusters<a id='5.2_Creating_a_new_feature_for_state_rent_clusters'></a> 

In [8]:
#Create three lists of state categories according to their median rent seen during exploratory data analysis
high_rent = ['Berlin', 'Hamburg']
medium_rent = ['Baden_Württemberg', 'Bayern', 'Hessen']
low_rent = ['Schleswig_Holstein', 'Thüringen', 'Niedersachsen', 'Saarland', 'Bremen', 'Sachsen',\
                  'Mecklenburg_Vorpommern', 'Sachsen_Anhalt', 'Brandenburg', 'Nordrhein_Westfalen', 'Rheinland_Pfalz']

In [9]:
#Create a dictionary for mapping state to rent cluster
rent_mapping = {}
for state in high_rent:
    rent_mapping[state] = 'high_rent_state'
    
for state in medium_rent:
    rent_mapping[state] = 'medium_rent_state'

for state in low_rent:
    rent_mapping[state] = 'low_rent_state'

rent_mapping

{'Berlin': 'high_rent_state',
 'Hamburg': 'high_rent_state',
 'Baden_Württemberg': 'medium_rent_state',
 'Bayern': 'medium_rent_state',
 'Hessen': 'medium_rent_state',
 'Schleswig_Holstein': 'low_rent_state',
 'Thüringen': 'low_rent_state',
 'Niedersachsen': 'low_rent_state',
 'Saarland': 'low_rent_state',
 'Bremen': 'low_rent_state',
 'Sachsen': 'low_rent_state',
 'Mecklenburg_Vorpommern': 'low_rent_state',
 'Sachsen_Anhalt': 'low_rent_state',
 'Brandenburg': 'low_rent_state',
 'Nordrhein_Westfalen': 'low_rent_state',
 'Rheinland_Pfalz': 'low_rent_state'}

In [10]:
#Create new feature - state_rent_class - for each observation
rentals['state_rent_class'] = rentals['state'].replace(rent_mapping)
rentals.loc[:, ['state', 'rent', 'state_rent_class']].sample(5)

,state,rent,state_rent_class
105293,Sachsen,432.00,low_rent_state
239913,Sachsen_Anhalt,337.82,low_rent_state
222769,Sachsen_Anhalt,274.00,low_rent_state
240309,Sachsen_Anhalt,560.00,low_rent_state
33625,Nordrhein_Westfalen,440.00,low_rent_state


# 6. Create a Dataframe For  Features and Target Variable<a id='5.6._Create_a_Dataframe_for_the_features_and_target_variable'></a> 

In [11]:
'''Create a features datafram and drop high cardinality (zip_code), low variance (telekomTvOffer & 'telekomUploadSpeed') 
, and redundant (yearConstructed) features. Also drop the target variable'''
X = rentals.drop(columns = ['rent', 'yearConstructed', 'telekomUploadSpeed', 'rent_per_sqm', 'telekomTvOffer', 'zip_code'])
X.head()

,state,serviceCharge,heatingType,newlyConst,balcony,firingTypes,hasKitchen,cellar,livingSpace,condition,...,population_2019,population_per_km2,gdp_per_capita_2018,hdi,total_state_listings,total_state_sqm,listings_per_1000capita,listings_per_100sqm,yearBuilt,state_rent_class
0,Nordrhein_Westfalen,245.0,central_heating,False,False,oil,False,True,86.00,well_kept,...,17932651,526,39678,0.936,62069,4615660.71,3.461228,1.344748,1965.0,low_rent_state
1,Nordrhein_Westfalen,95.0,self_contained_central_heating,False,False,gas,False,True,60.00,well_kept,...,17932651,526,39678,0.936,62069,4615660.71,3.461228,1.344748,1953.0,low_rent_state
2,Nordrhein_Westfalen,200.0,central_heating,False,False,oil,False,False,123.44,first_time_use_after_refurbishment,...,17932651,526,39678,0.936,62069,4615660.71,3.461228,1.344748,1951.0,low_rent_state
3,Nordrhein_Westfalen,215.0,gas_heating,True,True,gas,False,True,87.00,first_time_use,...,17932651,526,39678,0.936,62069,4615660.71,3.461228,1.344748,2018.0,low_rent_state
4,Nordrhein_Westfalen,121.0,central_heating,False,True,gas,False,True,65.00,well_kept,...,17932651,526,39678,0.936,62069,4615660.71,3.461228,1.344748,1914.0,low_rent_state


In [12]:
#Create a dataframe with the target variable
y = rentals[['rent']]
y.head()

,rent
0,595.0
1,300.0
2,950.0
3,972.6
4,329.0


# 7. Develop an Imputation Strategy<a id='7._Develop_an_Imputation_Strategy'></a>

In [13]:
# Review datatypes
X.dtypes

state                              object
serviceCharge                     float64
heatingType                        object
newlyConst                           bool
balcony                              bool
firingTypes                        object
hasKitchen                           bool
cellar                               bool
livingSpace                       float64
condition                          object
interiorQual                       object
petsAllowed                        object
lift                                 bool
typeOfFlat                         object
noRooms                           float64
thermalChar                       float64
numberOfFloors                    float64
garden                               bool
district                           object
town_municipality                  object
date                       datetime64[ns]
sqm_per_room                      float64
area_km2                            int64
population_2019                   

In [14]:
#Create a list of names for categorical features
cat_cols = X.select_dtypes('object').columns.to_list()
cat_cols

['state',
 'heatingType',
 'firingTypes',
 'condition',
 'interiorQual',
 'petsAllowed',
 'typeOfFlat',
 'district',
 'town_municipality',
 'state_rent_class']

In [15]:
#Create a list of names for numerical features
num_cols = X.select_dtypes(['float64', 'int64']).columns.to_list()
num_cols

['serviceCharge',
 'livingSpace',
 'noRooms',
 'thermalChar',
 'numberOfFloors',
 'sqm_per_room',
 'area_km2',
 'population_2019',
 'population_per_km2',
 'gdp_per_capita_2018',
 'hdi',
 'total_state_listings',
 'total_state_sqm',
 'listings_per_1000capita',
 'listings_per_100sqm',
 'yearBuilt']

In [16]:
#Use median imputation for numerical features
imputations = []
imputations.extend([([numeric_col], SimpleImputer(missing_values = np.nan, strategy = 'median')) \
                                for numeric_col in num_cols])
#Use 'most_frequent' imputation for categorical features
imputations.extend([([cat_col], SimpleImputer(strategy = 'most_frequent')) for cat_col in cat_cols])
imputations

[(['serviceCharge'], SimpleImputer(strategy='median')),
 (['livingSpace'], SimpleImputer(strategy='median')),
 (['noRooms'], SimpleImputer(strategy='median')),
 (['thermalChar'], SimpleImputer(strategy='median')),
 (['numberOfFloors'], SimpleImputer(strategy='median')),
 (['sqm_per_room'], SimpleImputer(strategy='median')),
 (['area_km2'], SimpleImputer(strategy='median')),
 (['population_2019'], SimpleImputer(strategy='median')),
 (['population_per_km2'], SimpleImputer(strategy='median')),
 (['gdp_per_capita_2018'], SimpleImputer(strategy='median')),
 (['hdi'], SimpleImputer(strategy='median')),
 (['total_state_listings'], SimpleImputer(strategy='median')),
 (['total_state_sqm'], SimpleImputer(strategy='median')),
 (['listings_per_1000capita'], SimpleImputer(strategy='median')),
 (['listings_per_100sqm'], SimpleImputer(strategy='median')),
 (['yearBuilt'], SimpleImputer(strategy='median')),
 (['state'], SimpleImputer(strategy='most_frequent')),
 (['heatingType'], SimpleImputer(strateg

In [17]:
#Create pipeline element that enables imputation of missing values
df_imputation = DataFrameMapper(imputations, input_df = True, df_out = True)
df_imputation

DataFrameMapper(df_out=True, drop_cols=[],
                features=[(['serviceCharge'], SimpleImputer(strategy='median')),
                          (['livingSpace'], SimpleImputer(strategy='median')),
                          (['noRooms'], SimpleImputer(strategy='median')),
                          (['thermalChar'], SimpleImputer(strategy='median')),
                          (['numberOfFloors'],
                           SimpleImputer(strategy='median')),
                          (['sqm_per_room'], SimpleImputer(strategy='m...
                           SimpleImputer(strategy='most_frequent')),
                          (['petsAllowed'],
                           SimpleImputer(strategy='most_frequent')),
                          (['typeOfFlat'],
                           SimpleImputer(strategy='most_frequent')),
                          (['district'],
                           SimpleImputer(strategy='most_frequent')),
                          (['town_municipality'],
      

# 8. Create Train-Test Split<a id='8._Create_Train-Test_Split'></a>

In [18]:
#Split X and y to test and training set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 77)

# 9. Create One Hot Encoding Object<a id='9._Create_One_Hot_Encoding_Object'></a>

In [19]:
onehot = OneHotEncoder(handle_unknown = 'ignore')

# 10. Create Scaling Object<a id='10._Create_Scaling_Object'></a>

In [20]:
scaler = StandardScaler(with_mean = False)

# 11. Modelling<a id = '11._Modelling'></a>

## 11.1 Linear Regression Model

In [21]:
#Instantiate linear regression model
lm = LinearRegression(normalize = False, random_state = 77)

In [23]:
#Create pipeline of pre-processing steps and model creation
steps_lm = [('imputation', df_imputation), ('encoding', onehot), ('scaling', scaler), ('lm', lm)]
pipeline_lm = Pipeline(steps_lm)

In [24]:
#Fit the linear regression model to the data
pipeline_lm.fit(X_train, y_train)

Pipeline(steps=[('imputation',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['serviceCharge'],
                                            SimpleImputer(strategy='median')),
                                           (['livingSpace'],
                                            SimpleImputer(strategy='median')),
                                           (['noRooms'],
                                            SimpleImputer(strategy='median')),
                                           (['thermalChar'],
                                            SimpleImputer(strategy='median')),
                                           (['numberOfFloors'],
                                            SimpleImputer(strategy='median')),
                                           (['sqm_per_ro...
                                            SimpleImputer(strategy='most_frequent')),
                                           (['district'],
    

In [25]:
#Determine predictions for train and the test set
y_train_pred_lm = pipeline_lm.predict(X_train)
y_test_pred_lm = pipeline_lm.predict(X_test)

In [26]:
#Define function to assess model performance based on R-squared, root mean squared error and mean absolute error
def train_test_performance(y_train, y_train_pred, y_test, y_test_pred):
    r2 = [r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)]
    rmse = [mean_squared_error(y_train, y_train_pred, squared = False), \
            mean_squared_error(y_test, y_test_pred, squared = False)]
    mae = [mean_absolute_error(y_train, y_train_pred), mean_absolute_error(y_test, y_test_pred)]
    df = pd.DataFrame({'r2_score': r2, 'rmse (€)': rmse, 'mae (€)': mae}, index = ['train', 'test'])
    df = df.append(df.iloc[[0, -1]].pct_change().iloc[-1].mul(100).rename('perf_change (%)'))
    return df

#Define function to assess cross-validated performance of learner
def cv_performance(model, X_train, y_train, n_folds = 4):
    cv_scores = cross_validate(model, X_train, y_train.values.ravel(), cv = n_folds, \
                              scoring = ['r2', 'neg_mean_squared_error','neg_mean_absolute_error'])
    r2_mean = np.mean(cv_scores['test_r2'])
    r2_std = np.std(cv_scores['test_r2'])
    rmse_mean = np.sqrt(np.mean(cv_scores['test_neg_mean_squared_error']) * -1)
    rmse_std = np.sqrt(np.std(cv_scores['test_neg_mean_squared_error']))
    mae_mean = np.mean(cv_scores['test_neg_mean_absolute_error']) * -1
    mae_std = np.std(cv_scores['test_neg_mean_absolute_error'])
    df = pd.DataFrame({'cv_r2':[r2_mean, r2_std], 'cv_rmse (€)': [rmse_mean, rmse_std], 'cv_mae (€)': [mae_mean, mae_std]},\
                     index = ['mean', 'std'])                                           
    return df

In [27]:
#Check performance for linear model
perf_lm = train_test_performance(y_train, y_train_pred_lm, y_test, y_test_pred_lm)
perf_lm

,r2_score,rmse (€),mae (€)
train,0.921025,143.344614,75.381070
test,0.716940,283.213579,115.175225
perf_change (%),-22.158536,97.575320,52.790648


- We note a substantial r2 performance gap between our train and test set. This suggests overfitting.

In [28]:
#Check cross-validated performance for linear model to include the mean error and standard deviation around the mean error
cv_lm = cv_performance(pipeline_lm, X_train, y_train, 4)
cv_lm

,cv_r2,cv_rmse (€),cv_mae (€)
mean,0.763833,248.076350,119.985722
std,0.014840,74.101128,0.675675


In [44]:
#Error we should expect in our prediction
np.round((cv_lm.loc['mean','cv_mae (€)'] - cv_lm.loc['std','cv_mae (€)']),2), \
np.round((cv_lm.loc['mean','cv_mae (€)'] + cv_lm.loc['std','cv_mae (€)']), 2) 

(119.31, 120.66)

- We note that the cross-validated performance is better than the single test performance.
- We expect that for this model the mae is the appropriate metric. This implies that The linear model has a low bias but high variance.

## 11.2 Random Forest Model

In [29]:
#Instantiate random forest model
rf = RandomForestRegressor(n_estimators = 10, max_depth = 15, random_state = 77)

In [31]:
#Create pipeline of pre-processing steps and model creation
steps_rf = [('imputation', df_imputation), ('encoding', onehot), ('scaling', scaler), ('rf', rf)]
pipeline_rf = Pipeline(steps_rf)

In [32]:
#Fit random forest model to data
pipeline_rf.fit(X_train, y_train.values.ravel())

Pipeline(steps=[('imputation',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['serviceCharge'],
                                            SimpleImputer(strategy='median')),
                                           (['livingSpace'],
                                            SimpleImputer(strategy='median')),
                                           (['noRooms'],
                                            SimpleImputer(strategy='median')),
                                           (['thermalChar'],
                                            SimpleImputer(strategy='median')),
                                           (['numberOfFloors'],
                                            SimpleImputer(strategy='median')),
                                           (['sqm_per_ro...
                                           (['district'],
                                            SimpleImputer(strategy='most_frequent')),
    

In [33]:
#Get predictions of rent on the train and test set with the random forest learner
y_train_pred_rf = pipeline_rf.predict(X_train)
y_test_pred_rf = pipeline_rf.predict(X_test)

In [34]:
#Check single pass performance of random forest model on train and test set
perf_rf = train_test_performance(y_train, y_train_pred_rf, y_test, y_test_pred_rf)
perf_rf

,r2_score,rmse (€),mae (€)
train,0.773552,242.728954,153.377393
test,0.605676,334.273059,162.416797
perf_change (%),-21.702035,37.714539,5.893570


- Based on the r2 score, we note that the ridge learner shows a lower bias when compared to random forest

In [36]:
#Check cross-validation performance of random forest model
cv_rf = cv_performance(pipeline_rf, X_train, y_train, 4)
cv_rf

,cv_r2,cv_rmse (€),cv_mae (€)
mean,0.657239,298.607428,163.084584
std,0.009092,59.891835,1.017414


- We also note that based on the r2 score, the random forest model has a lower cross-validation score than the ridge model.

## 11.3 Ridge Regression Model 

In [45]:
#Instantiate lasso model
ridge = Ridge(random_state = 77)

In [49]:
#Create pipeline of pre-processing steps and model creation
steps_ridge = [('imputation', df_imputation), ('encoding', onehot), ('scaling', scaler), ('ridge', ridge)]
pipeline_ridge = Pipeline(steps_ridge)

In [50]:
pipeline_ridge.fit(X_train, y_train.values.ravel())

Pipeline(steps=[('imputation',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['serviceCharge'],
                                            SimpleImputer(strategy='median')),
                                           (['livingSpace'],
                                            SimpleImputer(strategy='median')),
                                           (['noRooms'],
                                            SimpleImputer(strategy='median')),
                                           (['thermalChar'],
                                            SimpleImputer(strategy='median')),
                                           (['numberOfFloors'],
                                            SimpleImputer(strategy='median')),
                                           (['sqm_per_ro...
                                            SimpleImputer(strategy='most_frequent')),
                                           (['district'],
    

In [51]:
#Get predictions of rent from X_train and X_test
y_train_pred_ridge = pipeline_ridge.predict(X_train)
y_test_pred_ridge = pipeline_ridge.predict(X_test)

In [52]:
#Check single pass performance on train and test sets for ridge regression model
perf_ridge = train_test_performance(y_train, y_train_pred_ridge, y_test, y_test_pred_ridge)
perf_ridge

,r2_score,rmse (€),mae (€)
train,0.920948,143.415194,75.643419
test,0.717885,282.740391,114.910647
perf_change (%),-22.049341,97.148143,51.910964


In [53]:
#Check 4-fold cross-validation performance of ridge regression model
cv_ridge = cv_performance(pipeline_ridge, X_train, y_train, 4)
cv_ridge

,cv_r2,cv_rmse (€),cv_mae (€)
mean,0.765298,247.302564,119.528148
std,0.014509,73.455340,0.652628


# 12. Assessing Models by Pre-selecting Features<a id = '12._Assessing_Models_by_Pre-selecting_Features'></a>

## 12.1 Pre-selecting Features for Ridge Regression 

### 12.1.1 Pre-selecting a percentage of top features

In [54]:
#Instantiate object to select the top 5% of features using the f_regression test
selectP = SelectPercentile(score_func = f_regression, percentile = 5)

In [55]:
#Build pipeline to pre-select features for our ridge regression learner
selectP_steps_ridge = [('imputation', df_imputation), ('encoding', onehot), ('scaling', scaler), ('selectP', selectP),\
                   ('ridge', ridge)]
pipeline_selectP_ridge = Pipeline(selectP_steps_ridge)

In [56]:
#Fit pipeline to data
pipeline_selectP_ridge.fit(X_train, y_train.values.ravel())

Pipeline(steps=[('imputation',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['serviceCharge'],
                                            SimpleImputer(strategy='median')),
                                           (['livingSpace'],
                                            SimpleImputer(strategy='median')),
                                           (['noRooms'],
                                            SimpleImputer(strategy='median')),
                                           (['thermalChar'],
                                            SimpleImputer(strategy='median')),
                                           (['numberOfFloors'],
                                            SimpleImputer(strategy='median')),
                                           (['sqm_per_ro...
                                           (['town_municipality'],
                                            SimpleImputer(strategy='most_frequent

In [57]:
#Get performance of ridge learner with the top 5% of pre-selected features
y_train_pred_selectP_ridge = pipeline_selectP_ridge.predict(X_train)
y_test_pred_selectP_ridge = pipeline_selectP_ridge.predict(X_test)

In [58]:
#Check single pass performance for the ridge regression learner of pre-selected features
perf_ridge_selectP = train_test_performance(y_train, y_train_pred_selectP_ridge, y_test, y_test_pred_selectP_ridge)
perf_ridge_selectP

,r2_score,rmse (€),mae (€)
train,0.859542,191.165957,118.870612
test,0.711453,285.945031,126.180776
perf_change (%),-17.228796,49.579473,6.149681


- We observe less variance in r2 score when we pre-select features versus when features are not pre-selected

In [59]:
#Check 4-fold cross-validation performance of ridge regression learner with pre-selected features
cv_ridge_selectP = cv_performance(pipeline_selectP_ridge, X_train, y_train, 4)
cv_ridge_selectP

,cv_r2,cv_rmse (€),cv_mae (€)
mean,0.760466,249.850983,128.372755
std,0.015606,75.700998,0.440314


- We note that the learner's performance appears to improve with cross-validation, further reducing the initial variance observed with the single pass performance assessment

### 12.1.2 Pre-selecting a certain number of top features

In [60]:
#Instantiate a selection object based on f_regression that selects the top 40 features
selectK = SelectKBest(score_func = f_regression, k = 40)

In [61]:
#Build pipeline with selected features
selectK_steps_ridge = [('imputation', df_imputation), ('encoding', onehot), ('scaling', scaler), ('selectK', selectK),\
                   ('ridge', ridge)]
pipeline_selectK_ridge = Pipeline(selectK_steps_ridge)

In [62]:
#Fit pipeline to date
pipeline_selectK_ridge.fit(X_train, y_train.values.ravel())

Pipeline(steps=[('imputation',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['serviceCharge'],
                                            SimpleImputer(strategy='median')),
                                           (['livingSpace'],
                                            SimpleImputer(strategy='median')),
                                           (['noRooms'],
                                            SimpleImputer(strategy='median')),
                                           (['thermalChar'],
                                            SimpleImputer(strategy='median')),
                                           (['numberOfFloors'],
                                            SimpleImputer(strategy='median')),
                                           (['sqm_per_ro...
                                            SimpleImputer(strategy='most_frequent')),
                                           (['town_municipalit

In [63]:
#Get model predictions of rent based on pre-selecting the top 40 features
y_train_pred_selectK_ridge = pipeline_selectK_ridge.predict(X_train)
y_test_pred_selectK_ridge = pipeline_selectK_ridge.predict(X_test)

In [64]:
#Check single pass performance on the train and test sets for our ridge learner with 40 pre-selected features
perf_ridge_selectK = train_test_performance(y_train, y_train_pred_selectK_ridge, y_test, y_test_pred_selectK_ridge)
perf_ridge_selectK

,r2_score,rmse (€),mae (€)
train,0.467719,372.141279,216.110391
test,0.424834,403.711315,215.991609
perf_change (%),-9.168872,8.483347,-0.054964


- We note that our ridge learner performance (r2-score) worse when we pre-select 40 features on both the train and test sets
- We observe that our ridge learner explains less than 50% of the variation seen in our data

In [65]:
#Check 4-fold cross-validation performance of ridge learner with 40 pre-selected features
cv_ridge_selectK = cv_performance(pipeline_selectK_ridge, X_train, y_train, 4)
cv_ridge_selectK

,cv_r2,cv_rmse (€),cv_mae (€)
mean,0.464078,373.463911,216.755243
std,0.009670,79.252679,1.766775


- Our cross-validation performance by pre-selecting 40 features is comparatively less than observed when we pre-selected 5% of features.

## 12.2 Pre-selecting Features for Random Forest Model 

In [185]:
#Instantiate a selection object based on f_regression that selects the top 40 features
selectK = SelectKBest(score_func = f_regression, k = 40)

In [186]:
#Instantiate new random forest model
rf_selectK = RandomForestRegressor(random_state = 77)

In [187]:
#Set up pipeline for random forest learner and pre-selecting the top 40 features
selectK_steps_rf = [('imputation', df_imputation), ('encoding', onehot), ('scaling', scaler), ('selectK', selectK),\
                   ('rf_selectK', rf_selectK)]
pipeline_selectK_rf = Pipeline(selectK_steps_rf)

In [188]:
#Fit pipeline to data
pipeline_selectK_rf.fit(X_train, y_train.values.ravel())

Pipeline(steps=[('imputation',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['serviceCharge'],
                                            SimpleImputer(strategy='median')),
                                           (['livingSpace'],
                                            SimpleImputer(strategy='median')),
                                           (['noRooms'],
                                            SimpleImputer(strategy='median')),
                                           (['thermalChar'],
                                            SimpleImputer(strategy='median')),
                                           (['numberOfFloors'],
                                            SimpleImputer(strategy='median')),
                                           (['sqm_per_ro...
                                            SimpleImputer(strategy='most_frequent')),
                                           (['state_rent_class

In [189]:
#Get rent predictions for random forest learner with top 40 features pre-selected
y_train_pred_selectK_rf = pipeline_selectK_rf.predict(X_train)
y_test_pred_selectK_rf = pipeline_selectK_rf.predict(X_test)

In [190]:
#Check single pass train / test performance for random forest model with top 40 features pre-selected
perf_rf_selectK = train_test_performance(y_train, y_train_pred_selectK_rf, y_test, y_test_pred_selectK_rf)
perf_rf_selectK

,r2_score,rmse (€),mae (€)
train,0.510153,356.999615,206.733366
test,0.451697,394.171243,208.281970
perf_change (%),-11.458530,10.412232,0.749083


In [193]:
#Check 4-fold cross-validation performance of random forest learner with 40 pre-selected features
cv_rf_selectK = cv_performance(pipeline_selectK_rf, X_train, y_train, 4)
cv_rf_selectK

,cv_r2,cv_rmse (€),cv_mae (€)
mean,0.490873,363.967723,208.985597
std,0.010245,73.875372,1.631288


# 13 Hyper parameter tuning on ridge learner<a id = '13._Hyper_parameter_tuning_on_ridge_learner'></a> 

In [195]:
#Instantiate the selection of a percentage of features
selectP_search = SelectPercentile(score_func = f_regression)

In [196]:
#Create pipeline for our ridge learner
selectP2_steps_ridge = [('imputation', df_imputation), ('encoding', onehot), ('scaling', scaler), \
                        ('selectP_search', selectP_search), ('ridge_P', ridge)]
pipeline_selectP2_ridge = Pipeline(selectP2_steps_ridge)

In [197]:
#Specify parameters to assess in optimizing our ridge learner's performance
cv_ridge_params = {'selectP_search__percentile':np.arange(1, 10, 11), 'ridge_P__alpha': np.linspace(0, 1, 11)}

In [198]:
#Create object to search for optimum parameters
cv_ridge = GridSearchCV(estimator = pipeline_selectP2_ridge, param_grid = cv_ridge_params,
                        scoring = ['r2', 'neg_mean_squared_error','neg_mean_absolute_error'], refit = 'r2', cv = 4)

In [199]:
#Fit parameter search object to data
cv_ridge.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('imputation',
                                        DataFrameMapper(df_out=True,
                                                        drop_cols=[],
                                                        features=[(['serviceCharge'],
                                                                   SimpleImputer(strategy='median')),
                                                                  (['livingSpace'],
                                                                   SimpleImputer(strategy='median')),
                                                                  (['noRooms'],
                                                                   SimpleImputer(strategy='median')),
                                                                  (['thermalChar'],
                                                                   SimpleImputer(strategy='median')),
                                              

In [200]:
#Get the best parameters
cv_ridge.best_params_

{'ridge_P__alpha': 0.30000000000000004, 'selectP_search__percentile': 1}

In [201]:
#Get the predictions on our train and test sets for our ridge learner with optimal parameters
y_train_pred_cv_ridge = cv_ridge.predict(X_train)
y_test_pred_cv_ridge = cv_ridge.predict(X_test)

In [202]:
#Check single pass performance of ridge learner on train and test sets
perf_cv_ridge = train_test_performance(y_train, y_train_pred_cv_ridge, y_test, y_test_pred_cv_ridge)
perf_cv_ridge

,r2_score,rmse (€),mae (€)
train,0.777329,240.696228,145.257782
test,0.664015,308.556244,147.676454
perf_change (%),-14.577325,28.193219,1.665090


- On a single pass our ridge regression with optimized parameters performs worse than without optimized parameters. However, we observe less variance in results, suggesting less fitting of the model. 

## 13.1 Obtaining cross-validation scores from grid search

In [132]:
def search_performance(cv_object):
    metrics = ['mean_test_r2', 'std_test_r2', \
           'mean_test_neg_mean_squared_error', 'std_test_neg_mean_squared_error', \
          'mean_test_neg_mean_absolute_error', 'std_test_neg_mean_absolute_error']
    metrics_df = pd.DataFrame(cv_object.cv_results_).sort_values(by = 'rank_test_r2').reset_index().loc[[0], metrics]
    metrics_df['mean_test_neg_mean_squared_error'] = (metrics_df['mean_test_neg_mean_squared_error'] * -1).apply(np.sqrt)
    metrics_df['std_test_neg_mean_squared_error'] = (metrics_df['std_test_neg_mean_squared_error']).apply(np.sqrt)
    metrics_df['mean_test_neg_mean_absolute_error'] = metrics_df['mean_test_neg_mean_absolute_error'] * -1
    metrics_df_T = metrics_df.T
    metrics_df_T['measure'] = ['mean', 'std', 'mean', 'std', 'mean', 'std']
    metrics_df_T['metric'] = ['cv_r2', 'cv_r2', 'cv_rmse(€)', 'cv_rmse(€)', 'cv_mae(€)', 'cv_mae(€)']
    cv_search = pd.DataFrame((metrics_df_T.pivot(index = 'measure', columns = 'metric', values = 0)))
    cv_search = cv_search.loc[:, ['cv_r2','cv_rmse(€)','cv_mae(€)']]
    return cv_search

In [133]:
cv_ridge_search = search_performance(cv_ridge)
cv_ridge_search

metric,cv_r2,cv_rmse(€),cv_mae(€)
measure,,,
mean,0.708505,275.562473,150.496598
std,0.015652,77.774885,0.749103


- Our cross-validation scores show less variance compared to the single pass performance

# 14. XGBoost Model<a id = '14._XGBoost_Model'></a>

In [152]:
#Instantiate XGBoost learner
xg = xgb.XGBRegressor(objective = 'reg:squarederror', seed = 77)

In [153]:
#Create pipeline for xg boost learner
steps_xg = [('imputation', df_imputation), ('encoding', onehot), ('scaling', scaler), ('xg', xg)]
pipeline_xg = Pipeline(steps_xg)

In [154]:
#Fit pipeline to data
pipeline_xg.fit(X_train, y_train.values.ravel())

Pipeline(steps=[('imputation',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['serviceCharge'],
                                            SimpleImputer(strategy='median')),
                                           (['livingSpace'],
                                            SimpleImputer(strategy='median')),
                                           (['noRooms'],
                                            SimpleImputer(strategy='median')),
                                           (['thermalChar'],
                                            SimpleImputer(strategy='median')),
                                           (['numberOfFloors'],
                                            SimpleImputer(strategy='median')),
                                           (['sqm_per_ro...
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                        

In [155]:
#Get single pass rent predictions for train and test sets with the xg boost learner
y_train_pred_xg = pipeline_xg.predict(X_train)
y_test_pred_xg = pipeline_xg.predict(X_test)

In [156]:
#Check single pass performance of xg boost learner
perf_xg = train_test_performance(y_train, y_train_pred_xg, y_test, y_test_pred_xg)
perf_xg

,r2_score,rmse (€),mae (€)
train,0.838030,205.283534,130.743382
test,0.690263,296.258624,137.060332
perf_change (%),-17.632696,44.316798,4.831564


In [169]:
#Check 4-fold cross-validation performance of xg boost learner
cv_xg = cv_performance(pipeline_xg, X_train, y_train, 4)
cv_xg

,cv_r2,cv_rmse (€),cv_mae (€)
mean,0.749839,255.294341,137.400584
std,0.014260,73.735988,0.873913


## 14.1 Hyperparameter tuning on XGBoost learner 

In [176]:
#Set up search grid for XGBoost parameters
xg_params = {'xg__subsample': np.arange(0.1, 1, 0.1), 
                 'xg__max_depth':np.arange(3, 5, 1),
                 'xg__colsample_bytree': np.arange(0.1, 1, 0.1)}

In [177]:
#Set up randomized search with the xg boost learner over the search grid
search_xg = RandomizedSearchCV(estimator = pipeline_xg, param_distributions = xg_params, n_iter = 5, \
                            scoring = ['r2', 'neg_mean_squared_error','neg_mean_absolute_error'], refit = 'r2', cv = 4)

In [178]:
#Fit xg boost learner with search grid to data
search_xg.fit(X_train, y_train.values.ravel())

RandomizedSearchCV(cv=4,
                   estimator=Pipeline(steps=[('imputation',
                                              DataFrameMapper(df_out=True,
                                                              drop_cols=[],
                                                              features=[(['serviceCharge'],
                                                                         SimpleImputer(strategy='median')),
                                                                        (['livingSpace'],
                                                                         SimpleImputer(strategy='median')),
                                                                        (['noRooms'],
                                                                         SimpleImputer(strategy='median')),
                                                                        (['thermalChar'],
                                                                         SimpleImp

In [179]:
#Check xg boost learner's best parameters
search_xg.best_params_

{'xg__subsample': 0.4,
 'xg__max_depth': 4,
 'xg__colsample_bytree': 0.7000000000000001}

In [181]:
y_train_pred_search_xg = search_xg.predict(X_train)
y_test_pred_search_xg = search_xg.predict(X_test)

In [182]:
perf_xg_search = train_test_performance(y_train, y_train_pred_search_xg, y_test, y_test_pred_search_xg)
perf_xg_search

,r2_score,rmse (€),mae (€)
train,0.777118,240.810366,150.647774
test,0.656519,311.979563,152.862170
perf_change (%),-15.518775,29.554042,1.469916


In [184]:
#Check performance of xg boost learner with search grid
cv_xg_search = search_performance(search_xg)
cv_xg_search

metric,cv_r2,cv_rmse(€),cv_mae(€)
measure,,,
mean,0.70728,276.095890,153.618408
std,0.01212,71.873054,0.951187


# 15. Model Selection<a id = '15._Model_Selection'></a>

- We select the ridge regression model with gridsearch because it has a similar r-squared value as XGBoost (0.7), but with less variance in the mean absolute error - (153 +/- 0.95 for XGBoost) and (150 +/- 0.75 for ridge). 

# 16. Save Best Model<a id = '16._Save_best_model'></a> 

In [276]:
best_model = cv_ridge.best_estimator_
best_model.version = '1.0'
best_model.pandas_version = pd.__version__
best_model.numpy_version = np.__version__
best_model.sklearn_version = str(sklearn.__version__)
best_model.X_columns = [col for col in X_train.columns]
best_model.build_datetime = dt.now()
    
modelpath = '..\models'
if not os.path.exists(modelpath):
    os.mkdir(modelpath)
rentalsmodel_path = os.path.join(modelpath, 'rentals_pricing_model.pkl')
if not os.path.exists(rentalsmodel_path):
    with open(rentalsmodel_path, 'wb') as f:
        pickle.dump(best_model, f)